# Exploration and visualization of Welm lab organoid data

In [ ]:
import colorcet as cc
import pandas as pd
import polars as pl
import numpy as np

from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Label
from bokeh.palettes import Category10, Category20
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap

from pathlib import Path
from scipy import stats

from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
output_notebook()

In [ ]:
data_folder = Path("../../../../CDRP-benchmarking-analysis/data/raw")
welm_data_folder = data_folder / "Welm"
welm_omics_folder = welm_data_folder / "omics"
welm_screen_folder = welm_data_folder / "screen"

In [ ]:
screen_id_map = pd.read_csv(welm_screen_folder / "idMAP.txt", sep="\t")
screen_id_map["Adjusted"] = screen_id_map["Adjusted"].map(
    lambda id: str(id).replace("-", "")
)
screen_id_map.columns = ["model_id_original", "model_id_adjusted"]

In [ ]:
file_fmt = str(welm_screen_folder / "Combined.Screen{}.gr50.scores.txt")

welm_screen_a_data = pd.read_csv(file_fmt.format("A"), sep="\t")
welm_screen_a_data["screen_id"] = "A"

welm_screen_b_data = pd.read_csv(file_fmt.format("B"), sep="\t")
welm_screen_b_data["screen_id"] = "B"

welm_screen_c_data = pd.read_csv(file_fmt.format("C"), sep="\t")
welm_screen_c_data["screen_id"] = "C"

In [ ]:
welm_screens = [welm_screen_a_data, welm_screen_b_data, welm_screen_c_data]
welm_screen_data = pd.concat(welm_screens).merge(
    screen_id_map, left_on="HCIid", right_on="model_id_adjusted", how="inner"
)

welm_screen_data.head()

In [ ]:
nan_vals = [np.nan, np.inf, -np.inf]
welm_screen_data = welm_screen_data[~welm_screen_data["IC50"].isin(nan_vals)]
welm_screen_data["LN_IC50"] = np.log(welm_screen_data["IC50"])
welm_screen_data["LN_EC50"] = np.log(welm_screen_data["EC50"])

groups = welm_screen_data.groupby("drug", group_keys=False)
ic50_zscores = groups.apply(lambda g: stats.zscore(g["LN_IC50"])).to_frame(
    name="ZSCORE_LN_IC50"
)
ec50_zscores = groups.apply(lambda g: stats.zscore(g["LN_EC50"])).to_frame(
    name="ZSCORE_LN_EC50"
)

welm_screen_data = welm_screen_data.join(ic50_zscores).join(ec50_zscores)
welm_screen_data.head()

In [ ]:
corr = stats.pearsonr(welm_screen_data["LN_IC50"], welm_screen_data["LN_EC50"])
print("ln(IC50) : ln(EC50)", corr)

corr = stats.pearsonr(
    welm_screen_data["ZSCORE_LN_IC50"], welm_screen_data["ZSCORE_LN_EC50"]
)
print("zscore ln(IC50) : zscore ln(EC50)", corr)

In [ ]:
welm_rnaseq_fpkm_file = (
    welm_omics_folder / "combined.FPKM.welmRNA.wide.cleaned.txt"
)
welm_rnaseq_fpkm = pd.read_csv(welm_rnaseq_fpkm_file, sep="\t")

welm_rnaseq_fpkm.head()

In [ ]:
welm_rnaseq_samples = welm_rnaseq_fpkm.columns[2:]
welm_rnaseq_sample_metadata = [[c, *c.split("_")] for c in welm_rnaseq_samples]
welm_rnaseq_sample_metadata = pd.DataFrame(
    welm_rnaseq_sample_metadata,
    columns=[
        "rna_sample_id",
        "model_id",
        "model_type",
        "model_passage",
        "model_organoid_day",
        "rna_seq_method",
        "rna_seq_accession",
    ],
)
welm_rnaseq_sample_metadata["has_rna"] = True
welm_rnaseq_sample_metadata.head()

In [ ]:
# convert to feature matrix with shape (n_samples, n_features)
welm_rnaseq_fpkm_t = welm_rnaseq_fpkm[welm_rnaseq_samples].transpose()
welm_rnaseq_genes = welm_rnaseq_fpkm["Approved symbol"].to_list()
welm_rnaseq_fpkm_t.columns = welm_rnaseq_genes
welm_rnaseq_fpkm_t = welm_rnaseq_fpkm_t.rename_axis(
    index="sample_id", columns="gene_id"
)

welm_rnaseq_log2fpkm_t: pd.DataFrame = np.log2(welm_rnaseq_fpkm_t + 1)
welm_rnaseq_log2fpkm_t.head()

In [ ]:
# reduce dimensionality for visualization of expression data

reducer = TSNE(n_components=2, random_state=41)

X = welm_rnaseq_log2fpkm_t.to_numpy()
X_scaled = StandardScaler().fit_transform(X)
X_reduced = reducer.fit_transform(X_scaled)

In [ ]:
X_df = (
    pd.DataFrame(
        X_reduced,
        index=welm_rnaseq_log2fpkm_t.index,
        columns=["tsne_1", "tsne_2"],
    )
    .reset_index()
    .merge(
        welm_rnaseq_sample_metadata,
        left_on="sample_id",
        right_on="rna_sample_id",
    )
)

X_df.head()

In [ ]:
n_clusters = len(X_df["model_id"].unique())
clustering = KMeans(n_clusters=n_clusters, n_init="auto", random_state=41).fit(
    X_scaled
)

X_df["cluster_label"] = clustering.labels_
X_df.head()

In [ ]:
print(metrics.rand_score(X_df["model_id"], X_df["cluster_label"]))
print(
    metrics.normalized_mutual_info_score(X_df["model_id"], X_df["cluster_label"])
)

In [ ]:
# only plot models with more than one sample

model_id_counts = X_df["model_id"].value_counts()
keep_model_ids = model_id_counts[model_id_counts > 1].index
X_df_plot = X_df[X_df["model_id"].isin(keep_model_ids)]
X_df_plot.head()

In [ ]:
source = ColumnDataSource(X_df_plot)

factors = X_df_plot["model_id"].unique()
cmap = dict(zip(factors, cc.glasbey_light))
cmap = factor_cmap(
    "model_id", palette=list(cmap.values()), factors=list(cmap.keys())
)

TOOLTIPS = [
    ("ID", "@model_id"),
    ("Type", "@model_type"),
    ("Passage", "@model_passage"),
    ("Seq Method", "@rna_seq_method"),
    ("Cluster", "@cluster_label"),
]

p = figure(
    width=1500,
    height=900,
    tools="hover",
    tooltips=TOOLTIPS,
    title="TSNE: Welm gene expression",
)

p.scatter(
    "tsne_1", "tsne_2", color=cmap, source=source, size=20, fill_alpha=0.6
)

p.xaxis.axis_label = "TSNE 1"
p.yaxis.axis_label = "TSNE 2"

show(p)